# Data prepartion
---
In this notebook, we're going to:
1. fetch fetch open source data from https://opendata-renewables.engie.com/ and 
2. preprocess it for for further analysis in the subsequent notebooks.

Open source data is made availabble by Engie group and is distributed under Open License version 2.0 published by Etalab.


## Step 1: Fetch Data

In [1]:
# declare variables for web scraping the data
URL = 'https://opendata-renewables.engie.com'
LOCAL_DATA = 'data'
URL_DIR = f'{LOCAL_DATA}/web_scrape'

# Set target locations in S3
BUCKET = 'l4e-demo'
PREFIX = 'wind-turbine/training_data'

In [2]:
from bs4 import BeautifulSoup
import zipfile
import pandas as pd

from utils import *

In [3]:
# create a wroking directory for web scraping operations
!mkdir $LOCAL_DATA
!mkdir $URL_DIR

# download index.html
!wget --output-document=$URL_DIR/index.html $URL

# scrape with BS
with open(f'{URL_DIR}/index.html') as fp:
    soup = BeautifulSoup(fp, "html.parser")

# get url paths to the raw zipped files
data_urls = [URL + x['href'].\
        replace('explore','media'). \
        replace('dataset','datasets'). \
        replace('/information','.zip') \
        for x in list(set([x for x in soup.find_all('a', href=True) if 'dataset' in x['href']]))]

# store url paths as a text
with open(f'{URL_DIR}/url_list.txt', 'w') as f:
    for item in data_urls:
        f.write("%s\n" % item)

# download raw zipped data files
!wget --input $URL_DIR/url_list.txt --directory-prefix=$URL_DIR/raw_zip

--2021-04-08 04:28:59--  https://opendata-renewables.engie.com/
Resolving opendata-renewables.engie.com (opendata-renewables.engie.com)... 94.143.221.195
Connecting to opendata-renewables.engie.com (opendata-renewables.engie.com)|94.143.221.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/web_scrape/index.html’

data/web_scrape/ind     [ <=>                ]  24.69K  --.-KB/s    in 0.09s   

2021-04-08 04:28:59 (284 KB/s) - ‘data/web_scrape/index.html’ saved [25280]

--2021-04-08 04:28:59--  https://opendata-renewables.engie.com/media/datasets/6eeb7f50-97f7-4ab2-8d36-c6d9f9491d84.zip
Resolving opendata-renewables.engie.com (opendata-renewables.engie.com)... 94.143.221.195
Connecting to opendata-renewables.engie.com (opendata-renewables.engie.com)|94.143.221.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456 [application/zip]
Saving to: ‘data/web_scrape/raw_zip/6eeb7f50-97f7-4ab2-8d36-c6

In [4]:
zip_files = !ls $URL_DIR/raw_zip
zip_files

['01c55756-5cd6-4f60-9f63-2d771bb25a1a.zip',
 '39490fd2-04a2-4622-9042-ce4dd34c2a58.zip',
 '6eeb7f50-97f7-4ab2-8d36-c6d9f9491d84.zip',
 'd543716b-368d-4c53-8fb1-55addbe8d3ad.zip']

In [5]:
# unzip files
for zip_file_ in zip_files:
    with zipfile.ZipFile(f'{URL_DIR}/raw_zip/{zip_file_}', 'r') as zip_ref:
        zip_ref.extractall(f'{URL_DIR}/raw_unzip')

In [6]:
# print unzipped files names
unzipped_files = !ls $URL_DIR/raw_unzip
unzipped_files

['data_description.csv',
 'la-haute-borne-data-2013-2016.csv',
 'la-haute-borne-data-2017-2020.csv',
 'static-information.csv']

## Step 2: Preprocess data

In [7]:
data_files = [x for x in unzipped_files if '-data-' in x]
data_files

['la-haute-borne-data-2013-2016.csv', 'la-haute-borne-data-2017-2020.csv']

In [8]:
# read
df_1 = pd.read_csv(f'{URL_DIR}/raw_unzip/{data_files[0]}', delimiter=';')
df_2 = pd.read_csv(f'{URL_DIR}/raw_unzip/{data_files[1]}', delimiter=';')
df_c = pd.concat([df_1 ,df_2], axis=0)

turbines = df_c['Wind_turbine_name'].unique()
print(f'List of turbines in the data: {turbines}')

List of turbines in the data: ['R80711' 'R80736' 'R80790' 'R80721']


In [43]:
for turbine in turbines:
    df_raw = df_c.loc[df_c['Wind_turbine_name'] == turbine,:]
    df_clean = clean_up_data(df_raw)
    df_description = pd.read_csv(f'{URL_DIR}/raw_unzip/data_description.csv', delimiter=';')
    df_ord = order_columns(df_clean, df_description)
    df_ord = df_ord.loc[:,~df_ord.columns.duplicated()]
    df_ord.to_csv(f's3://{BUCKET}/{PREFIX}/{turbine}/telemetry.csv')
    break

/home/ec2-user/SageMaker/lookout-for-equipment-demo/blogs/wind-turbine-engie/utils.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'] = pd.to_datetime(df['Date_time'], infer_datetime_format=True, utc=True)
/home/ec2-user/SageMaker/lookout-for-equipment-demo/blogs/wind-turbine-engie/utils.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['Timestamp'], keep='first', inplace=True)


In [45]:
# clean up
!rm -rf data